# ML4CPS Project-2 | NB-4

In [ ]:
import os
import sys

basepath = os.path.abspath(os.path.join(".."))
if not basepath in sys.path:
    sys.path.append(basepath)

%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from utils.dataset import DatasetLoaderXL, DataProcessor, LocationProcessor

## Load dataset

In [ ]:
dsxl = DatasetLoaderXL(dataset_dir="../dataset/raw", seed=42, verbose=True)
dsxl.load_all_datasets()

In [ ]:
dsxl.list_suburbs()

In [ ]:
dsxl.list_categories()

In [ ]:
dsxl.list_subcategories("Hospital", console_print=False);
dsxl.list_subcategories("Services", console_print=False);

In [ ]:
subcategories = [
    "Category 4 & 5 emergency department presentations",
    "Distance to nearest public hospital",
    "Distance to nearest public hospital with emergency department",
    "Distance to nearest public hospital with maternity services",
    "Presentations to emergency departments due to injury",
    "Presentations to emergency departments, 2012-13",
    "Public hospital separations, 2012-13",
    "Time to nearest public hospital with maternity services",
    "Travel time to nearest public hospital",
    "Travel time to nearest public hospital with emergency department",
]
df = dsxl.get_subcategories_across_all_suburbs(subcategories)

In [ ]:
subcategories = [
    "Aged Care (High Care)",
    "Aged Care (Low Care)",
    "Aged Care (SRS)",
    "Allied Health",
    "Alternative Health",
    "Bush Nursing Centres",
    "Centrelink Offices",
    "Child Protection and Family",
    "Community Health Centres",
    "Dental",
    "Disability",
    "General Practice",
    "Homelessness",
    "Kinder and/or Childcare",
    "Medicare Access Points",
    "Medicare Offices",
    "Mental Health",
    "Other Schools",
    "P12 Schools",
    "Pharmacies",
    "Primary Schools",
    "Private Hospitals",
    "Public Hospitals",
    "Secondary Schools",
]
df = dsxl.get_subcategories_across_all_suburbs(subcategories)
df.T

In [ ]:
dp = DataProcessor(df)
dp.normalize()
dp.get_topk_abs_correlations(k=10)

In [ ]:
dp.run_anova_analysis(subcategories)

In [ ]:
dp.plot_dendrogram()

In [ ]:
dfs = dp.get_similarity_matrix(metric="euclidean")

In [ ]:
dfs = dp.get_similarity_matrix(metric="cosine")

In [ ]:
dp.get_similar_suburbs(dfs, n_neighbours=3)

In [ ]:
dp.plot_similarity_heatmap(dfs);

In [ ]:
dfs = dp.get_similarity_matrix(metric="euclidean")
mds_df = dp.run_metric_mds_and_plot(dfs, isMetricMDS=True);
dp.plot_dendrogram(mds_df)

In [ ]:
locations = dsxl.get_subcategory_across_all_suburbs("Location")
lp = LocationProcessor(locations)
coordinates = lp.get_coordinates()
lp.calculate_proximity_matrix()

In [ ]:
coordinates = lp.reflect_coordinates(coordinates)
coordinates = lp.rotate_coordinates(coordinates, angle=90)
lp.plot_coordinates(coordinates)